In [ ]:
import smartcitizen_connector
import scdata as sc
import time
now = time.localtime()

In [ ]:
df = smartcitizen_connector.search_by_query(endpoint='devices',
                                            search_items=[{
                                                'key': 'postprocessing_id', 
                                                'value': 'not_null',
                                                'full': True
                                            }])

In [ ]:
devices = []
discard = [17327, 17370]
for item in df['postprocessing']:
    print (item['device_id'], df.loc[item['device_id'], 'name'], item['meta'])
    if item['device_id'] in discard: continue
    if item['meta'] == 'development-ajuntamentc40-1':
        devices.append(item['device_id'])

In [ ]:
devices

In [ ]:
test = sc.Test(name=f'C40_BATCH_1',
            devices=[sc.Device(blueprint='sc_air', params=sc.APIParams(id=id)) for id in devices]
)
test.options.cache=True

In [ ]:
test = sc.Test(name=f'C40_BATCH_1')
test.options.cache=True

In [ ]:
await test.load()

In [ ]:
traces = {
            "1": {"devices": "all",
                  "channel": "PMS5003_PM_1",
                  "subplot": 1}
        }

test.ts_uplot(traces = traces, 
              formatting = {'width':1000, 
                            'title': 'PM_1',
                            'padding-bottom': 600,
                            'ylabel': {1: 'PM1 (ug/cm3)'}
                           }, 
              options = {'frequency': '5Min'})

In [ ]:
traces = {
            "1": {"devices": "all",
                  "channel": "SCD30_CO2",
                  "subplot": 1}
        }

test.ts_uplot(traces = traces, 
              formatting = {'width':1000, 
                            'title': 'CO2',
                            'padding-bottom': 600,
                            'ylabel': {1: 'CO2 (ppm)'}
                           }, 
              options = {'frequency': '1Min'})

In [ ]:
traces = {
            "1": {"devices": "all",
                  "channel": "TEMP",
                  "subplot": 1}
        }

test.ts_uplot(traces = traces, 
              formatting = {'width':1000, 
                            'title': 'Temperature',
                            'padding-bottom': 600,
                            'ylabel': {1: 'Temp (degC)'}
                           }, 
              options = {'frequency': '1Min'})

In [ ]:
traces = {
            "1": {"devices": "all",
                  "channel": "NOISE_A",
                  "subplot": 1}
        }

test.ts_uplot(traces = traces, 
              formatting = {'width':1000, 
                            'title': 'Noise',
                            'padding-bottom': 600,
                            'ylabel': {1: 'Noise (dBA)'}
                           }, 
              options = {'frequency': '10Min'})

In [ ]:
traces = {
            "1": {"devices": "all",
                  "channel": "BATT",
                  "subplot": 1}
        }

test.ts_uplot(traces = traces, 
              formatting = {'width':1000, 
                            'title': 'Battery',
                            'padding-bottom': 600,
                            'ylabel': {1: 'Battery (%)'}
                           }, 
              options = {'frequency': '1Min'})

In [ ]:
traces = {
            "1": {"devices": "all",
                  "channel": "PRESS",
                  "subplot": 1}
        }

test.ts_uplot(traces = traces, 
              formatting = {'width':1000, 
                            'title': 'Pressure',
                            'ylabel': {1: 'Pressure (kPa)'}
                           }, 
              options = {'frequency': '1Min'})

### Data gaps

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import logging
import asyncio
import datetime
from scdata.tools.date import localise_date

In [ ]:
for device in test.devices:
    a = localise_date(datetime.datetime.now(), 'UTC') - device.handler.json.last_reading_at
    hours_ago = round(a.total_seconds() / 3600)
    print (device.id, device.handler.json.last_reading_at, hours_ago)
    plt.figure(figsize=(15,6))
    df = test.get_device(device.id).data.resample('5Min').mean()
    sns.heatmap(df.isna().transpose(),
                cmap="YlGnBu",
                cbar_kws={'label': 'Missing Data'}).set(title=f'{test.name}: {device.id} - ({device.handler.json.name}) {hours_ago}h')

In [ ]:
test.get_device(17512).sensors